  ## Libraries

In [1]:
# IMPORT THESE FIRST.
import numpy as np
import pandas as pd
from sklearn.neighbors import KDTree
#from sklearn.preprocessing import LabelEncoder
#from sklearn.model_selection import train_test_split
#import seaborn as sn
#import matplotlib.pyplot as plt
#from matplotlib.colors import ListedColormap
#from sklearn.model_selection import train_test_split
#from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
#from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
#from sklearn.datasets import (make_classification, make_blobs, load_breast_cancer, make_regression, make_friedman1,
#                              load_iris)
#from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
#from sklearn.svm import SVC, LinearSVC
#from sklearn.tree import DecisionTreeClassifier, plot_tree
#from sklearn.model_selection import cross_val_score, validation_curve, train_test_split

In [2]:
df = pd.read_excel("https://github.com/TechLabs-Berlin/wt21-pet-match/blob/main/data_set/pet-set.xlsx?raw=true")
#df.head()

### variable discribing the satisfaction level in percentage

In [3]:
# replacing strings with numerical values
df['expectations'] = df['My cat has met all my expectations'].replace({'Strongly agree': 5, 'Agree': 4, 'Neither agree nor disagree': 3, 'Disagree': 2, 'Strongly disagree': 1})
df['happy_owner'] = df['I am happy with my cat'].replace({'Strongly agree': 5, 'Agree': 4, 'Neither agree nor disagree': 3, 'Disagree': 2, 'Strongly disagree': 1})
df['happy_cat'] = df['I feel my cat is happy living with me'].replace({'Strongly agree': 5, 'Agree': 4, 'Neither agree nor disagree': 3, 'Disagree': 2, 'Strongly disagree': 1})
df['rehome'] = df['I have considered relinquishing or rehoming this cat to someone else'].replace({'Strongly agree': 1, 'Agree': 2, 'Neither agree nor disagree': 3, 'Disagree': 4, 'Strongly disagree': 5})

# aggregating final score
df['satisf_level'] = (df['expectations'] + df['happy_owner'] + df['happy_cat'] + df['rehome'])/20

# Owners / Adopters

The aggregarion was made by Hany, all comments also belong to him:)
Personality traits grouped in five scores according to BFI:

* Extraversion

* Agreeableness

* Conscientiousness

* Neuroticism  (how tense and moody the person )

* Creativity

In [4]:
# slice with the BFI relevant adopters data
adopters = df.iloc[:, 61:]
# remove the NaN , they are text here
adopters.replace("NaN",np.nan, inplace =True)
adopters.fillna(0,inplace=True)

# lisitng the Reverse questions (take care of the numbering)
rev_col = adopters.iloc[:,[1,5,7,8,11,17,20,22,23,26,30,33,34,36,40,42]]
rev_ques = rev_col.replace({'Agree strongly': 1, 'Agree a little': 2, 'Neither agree nor disagree': 3, 'Disagree a little': 4, 'Disagree strongly': 5})

#listing normal questions 
normal_ques = adopters.drop(columns = rev_col).replace({'Agree strongly': 5, 'Agree a little': 4, 'Neither agree nor disagree': 3, 'Disagree a little': 2, 'Disagree strongly': 1})

# make them into one set, 
adopt_score =  pd.concat([ normal_ques, rev_ques], axis=1)

#adopt_score.head()

#### note: the code may require some refactoring (working with the bigger set )
- **used "conctinate instade of inplace= True** : with .replace it gets warning " copy of a slice from a DataFrame" and the inplace is not fully happpening espically when applaying that for the 2nd time 

#### clastering (seems little hard coded, due to the use of concatinate)

- rules (all them are re-aggranged as follows)
Extraversion:(8 questions) 1, 6R, 11, 16, 21R, 26, 31R, 36
Agreeableness: (9 questions)2R, 7, 12R, 17, 22, 27R, 32, 37R, 42 ## high no = high agreebleness 
Conscientiousness: (9 questions)3, 8R, 13, 18R, 23R, 28, 33, 38, 43R
Neuroticism:(8 questions) 4, 9R, 14, 19, 24R, 29, 34R, 39
Openness: (10 ques )5, 10, 15, 20, 25, 30, 35R, 40, 41R, 44 

In [5]:
# arrange them for better reading.


extraversion = adopt_score.iloc[:,[0,29,6,10,34,16,38,22]]          ##Extraversion
agreeableness = adopt_score.iloc[:,[28,4,32,11,14,37,20,41,26]]     ##Agreeableness
conscientiousness = adopt_score.iloc[:,[1,30,7,33,35,17,21,23,43]]  ##Conscientiousness
neuroticism = adopt_score.iloc[:,[2,31,8,12,36,18,39,24]]                 ##Neuroticism  (how tense and moody the person )
creativity = adopt_score.iloc[:,[3,5,9,13,15,19,40,25,42,-1]]      ## openess & creativity


## putting them into columns with precenatges (it can be added to any df )

adopt_score["extraversion"] =extraversion.sum(axis=1)/(len(extraversion.columns)*5)
adopt_score["easy_going"] =agreeableness.sum(axis=1)/(len(agreeableness.columns)*5)
adopt_score["diligent"] =conscientiousness.sum(axis=1)/(len(conscientiousness.columns)*5)
adopt_score["moody_tense"] =neuroticism.sum(axis=1)/(len(neuroticism.columns)*5)
adopt_score["creative"] =creativity.sum(axis=1)/(len(creativity.columns)*5)

adopt_scores_only = adopt_score[["extraversion", "easy_going", "diligent", "moody_tense", "creative"]]

In [6]:
#adopt_scores_only.head(3)

# Cats

Grouping cats into 4 personalitiy-Cluster performed by Dayo, I changed it a bit so that we get comparable scores for cats and adopters.

* PC1: Gregariousness (confident, friendly, interactive

* PC: Aggressiveness (agonistic when handled or interacted with)

* PC: Aloofness/avoidance(avoidant of interaction, not friendly

* PC4: Anxiousness/fearfulness (wary of novel/less familiarpeople)


In [7]:
cats_pc = df.iloc[:, 26 : 54]
# remove the NaN , they are text here
cats_pc.replace("NaN",np.nan, inplace =True)
cats_pc.fillna(0,inplace=True)

# listing the Reverse questions (take care of the numbering)
rev_col_c = cats_pc.iloc[:,[9,16,18,19,20,27]].rename(columns = {"My cat comes and asks me for attention and initiates contact with me (e.g. the cat comes and sits on my knee, or rubs up against me and around me, in order to receive fuss/strokes/chin/cheek tickles)": 'REVERSED My cat comes and asks me for attention and initiates contact with me',
                                          'My cat is timid': 'REVERSED My cat is timid',
                                          'My cat likes being stroked': 'REVERSED My cat likes being stroked',
                                          'My cat is very tolerant to being handled': 'REVERSED My cat is very tolerant to being handled',
                                          'My cat is fearful': 'REVERSED My cat is fearful',
                                          'My cat would prefer to be left alone, rather than be with people': 'REVERSED My cat would prefer to be left alone, rather than be with people'})
rev_ques_c = rev_col_c.replace({'Strongly disagree': 5, 'Never': 5, 'Disagree': 4, 'Once': 4, 'Neither agree nor disagree': 3, 'Occasionally': 3, 'Agree': 2, 'Usually': 2, 'Strongly agree': 1, 'Always': 1})

#listing normal questions 
normal_ques_c = cats_pc.replace({'Strongly disagree': 1, 'Never': 1, 'Disagree': 2, 'Once': 2, 'Neither agree nor disagree': 3, 'Occasionally': 3, 'Agree': 4, 'Usually': 4, 'Strongly agree': 5, 'Always': 5}).drop(columns=cats_pc.iloc[:, [18, 19, 22]])

# make them into one set, 
cats_score =  pd.concat([normal_ques_c, rev_ques_c], axis=1)

# grouping
gregariousness = cats_score.iloc[:,[0,1,2,9,15,17,18,23,24,26,27]] #Gregariousness (confident, friendly, interactive)
aggressiveness = cats_score.iloc[:,[3,7,10,11,12,13,29]] #Aggressiveness (agonistic when handled or interacted with)
avoidance = cats_score.iloc[:,[4,5,6,8,14,19,25,28]] #Aloofness/avoidance(avoidant of interaction, not friendly)
anxiousness = cats_score.iloc[:,[16,20,21,22,24,]] #Anxiousness/fearfulness (wary of novel/less familiarpeople)

# aggregating to percentages
cats_score["friendly"] =gregariousness.sum(axis=1)/(len(gregariousness.columns)*5)
cats_score["agressive"] =aggressiveness.sum(axis=1)/(len(aggressiveness.columns)*5)
cats_score["avoidant"] =avoidance.sum(axis=1)/(len(avoidance.columns)*5)
cats_score["anxious"] =anxiousness.sum(axis=1)/(len(anxiousness.columns)*5)

cats_scores_only = cats_score[["friendly","agressive", "avoidant", "anxious"]]

In [8]:
df_scores = pd.concat([cats_scores_only, adopt_scores_only, df["satisf_level"]], axis=1)
#df_scores.head(3)

# Three datasets with scores only:

* adopt_scores_only: personality scores of potential adopters
* cats_scores_only: cats' personality scores
* df_scores: cats' and owners' scores + satisfaction level

In [12]:
#adopt_scores_only.head(5)
#cats_scores_only.head(5)
#df_scores.head(5)
#df_scores.isna().value_counts()

# KDTree, cats
clusters similar cats

X - cats_scores_only

In [13]:
df_1 = df_scores.dropna()

In [14]:
cats = df_1[['friendly', 'agressive', 'avoidant', 'anxious']]

In [15]:
# building a tree

tree = KDTree(cats, leaf_size = 2)

In [16]:
# returning the distance and indexes of the 10 neares neignbors
dist, ind = tree.query(cats[:1], k=10)                
print(1-dist, ind)

[[1.         0.96203519 0.94758908 0.94742966 0.94285714 0.94044067
  0.93961039 0.93817513 0.93394214 0.933543  ]] [[   0 1849 1737  240 3054  433 1446 3005  627 2018]]


In [17]:
# returning indices of the nearest cats

ind.tolist()[0]

[0, 1849, 1737, 240, 3054, 433, 1446, 3005, 627, 2018]

In [18]:
# transforming distances into percentages
dist = 1 - dist
(1-dist).tolist()[0]

[0.0,
 0.037964806474052315,
 0.05241092484405263,
 0.052570339357845386,
 0.05714285714285716,
 0.059559332178817925,
 0.06038961038961044,
 0.06182486594879755,
 0.06605785782326745,
 0.06645700220580597]

In [19]:
recomend_list = pd.DataFrame()
recomend_list['labels'] = ind.tolist()[0]
recomend_list['distance'] = (1-dist).tolist()[0]
recomend_list = recomend_list.set_index('labels')

In [22]:
# returns the indices and percentage
recomend_list

distance
labels          
0       0.000000
1849    0.037965
1737    0.052411
240     0.052570
3054    0.057143
433     0.059559
1446    0.060390
3005    0.061825
627     0.066058
2018    0.066457

In [23]:
#returning nearest cats personalities (seem random:)
pd.concat([cats.loc[ind.tolist()[0]], recomend_list], axis = 1 )

friendly  agressive  avoidant  anxious  distance
0     0.690909   0.314286     0.650     0.68  0.000000
1849  0.600000   0.628571     0.625     0.68  0.037965
1737  0.636364   0.485714     0.425     0.84  0.052411
240   0.763636   0.342857     0.350     0.64  0.052570
3054  0.672727   0.400000     0.450     0.56  0.057143
433   0.672727   0.400000     0.875     0.72  0.059559
1446  0.854545   0.200000     0.375     0.60  0.060390
3005  0.600000   0.542857     0.575     0.76  0.061825
627   0.727273   0.342857     0.575     0.60  0.066058
2018  0.927273   0.200000     0.450     0.64  0.066457

# KDTree Adopters

In [24]:
adopters = df_1[['extraversion',	'easy_going',	'diligent',	'moody_tense',	'creative']]
# building a tree
tree_a = KDTree(adopters, leaf_size = 2)

extraversion  easy_going  diligent  moody_tense  creative
0            0.675    0.822222  0.711111        0.700      0.64
1            0.675    0.800000  0.800000        0.750      0.78
2            0.625    0.418889  0.466667        0.600      0.28
3            0.575    0.800000  0.577778        0.925      0.72
4            0.575    0.733333  0.666667        0.675      0.58
...            ...         ...       ...          ...       ...
3326         0.800    0.800000  0.933333        0.450      0.70
3327         0.700    0.755556  0.800000        0.425      0.80
3328         0.550    0.733333  0.555556        0.825      0.80
3329         0.675    0.733333  0.822222        0.750      0.70
3330         0.425    0.755556  0.711111        0.700      0.62

[3248 rows x 5 columns]

# Return nearest cats

#### The following part was made to check if I can do something at all. This all needs to be replaced and use the whole adopter-cat raw to analize the distances, so that the whole step of finding nearest adopter and than nearest cats for this person cat might be undone. How i see it:
* we use the whole raw with all the data on cats and owners and make a regression to predict satisfaction level (e.g. predicted satisfaction level in percentages)
* for the potential adopter we make CROSS JOIN with all the cats we have, and predict the satisfaction level for each pair

In [26]:
def cat_recommender(adopter):
  # takes the characteristics of test adopter and finds the nearest neighbour in the training set, returns the distance and index of the nearest adopter from the training set
  dist_a, ind_a = tree_a.query(adopter, k=1)
  # takes the cat matching the adopter from the training set (the cat and adopter have the same indices)
  dist, ind = tree.query(cats.loc[ind_a.tolist()[0]], k=10) 
  reccomend_list = pd.DataFrame()
  reccomend_list['labels'] = ind.tolist()[0]
  reccomend_list['distance'] = (1-dist).tolist()[0]
  reccomend_list = reccomend_list.set_index('labels')
  #returning nearest cats, their personality scores and distance in percentages
  return pd.concat([cats.loc[ind.tolist()[0]], reccomend_list], axis = 1 )

In [27]:
cat_recommender([[0.75, 0.3, 0.25, 0.9, 0.1]])

friendly  agressive  avoidant  anxious  distance
2     0.654545   0.514286     0.700     0.64  1.000000
879   0.781818   0.228571     0.375     0.48  0.955872
3069  0.727273   0.485714     0.525     0.88  0.946797
2083  0.818182   0.314286     0.350     0.48  0.938425
1182  0.654545   0.228571     0.325     0.84  0.927564
406   0.727273   0.371429     0.425     0.56  0.927564
125   0.709091   0.228571     0.350     0.64  0.927564
2016  0.763636   0.200000     0.450     0.56  0.923096
477   0.763636   0.228571     0.550     0.64  0.921924
1931  0.636364   0.314286     0.375     0.96  0.914179